In [57]:
import ee
ee.Initialize()

DEM = ee.Image("USGS/SRTMGL1_003") # elevation map

#node1 = [-72, -42]
#node2 = [-73, -42]
#node3 = [-72, -43]
#node4 = [103, 3]

#nodes = [node1, node2, node3, node4]

import pandas as pd
df = pd.read_csv('regression_data.csv')
nodes = []

for i in range(0, 2000):
    temp = [df['longitude'][i], df['latitude'][i]]
    nodes.append(temp)

# make points from nodes
points = [ee.Geometry.Point(coord) for coord in nodes]

# make features from points (name by list order)
feats = [ee.Feature(p, {'name': 'node{}'.format(i)}) for i, p in enumerate(points)]

# make a featurecollection from points
fc = ee.FeatureCollection(feats)

# extract points from DEM
reducer = ee.Reducer.first()
data = DEM.reduceRegions(fc, reducer.setOutputs(['elevation']), 30)

# see data
#for feat in data.getInfo()['features']:
    # print(feat['properties'])

# export as CSV
new1 = []
for i in range(0, 2000):
    new = data.getInfo()['features'][i]['properties']['elevation']
    new1.append(new)
new1

KeyboardInterrupt: 

In [54]:
# change file name for each export
new1 = pd.DataFrame(new1, columns = ['altitude'])
new1.to_csv('batch1.csv',index=False)